# Exploration 4 - 멋진 인공지능 작사가 만들기 🎵

### Step 1. 라이브러리 불러오기

In [105]:
import os, re 
import numpy as np
import tensorflow as tf

### Step 2. 데이터 읽어오기
- glob 를 활용하여 모든 txt 파일을 읽어온 후, raw_corpus 리스트에 문장 단위로 저장하기

In [106]:
import glob

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()  # 행(줄) 단위로 담기
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


### Step 3. 데이터 정제하기
<!-- 
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요! -->

토큰화하기: 정규표현식(Regex)을 이용 `import re`
1. 문장 부호 양쪽에 공백을 추가 
2.모든 문자들을 소문자로 변환  
3.특수문자들은 모두 제거 

In [108]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기

In [109]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
    # 토큰의 개수가 15개를 넘어가는 문장은 제외합니다.
    if len(preprocessed_sentence.split()) > 15: continue
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

벡터화(vectorize)하기
- tf.keras.preprocessing.text.Tokenizer 패키지는 정제된 데이터를 토큰화하고, 데이터를 숫자로 변환. 
- 숫자로 변환된 데이터: 텐서(tensor)

####  단어장의 크기는 12,000 이상 으로 설정
#### 문장 길이 15로 제한: maxlen 사용

In [ ]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer(단어장)를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다 - 문자 데이터 입력 받아 리스트 형태로 반환
    tokenizer.fit_on_texts(corpus)
    
    # 준비한 tokenizer를 이용해 corpus를 Tensor(시퀀스)로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15) # maxlen 사용하여 문장길이 제한 
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

표준정규분포에서 95% 확률 이내로 포함하게 하는 식 사용: mean + 2*std
- 원하는 문장 길이가 나오지 않아 사용하지 않기로 함...pass

In [131]:
# # 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# # 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# # https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# # https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
# def tokenize(corpus):
#     # 12000단어를 기억할 수 있는 tokenizer(단어장)를 만들겁니다
#     # 우리는 이미 문장을 정제했으니 filters가 필요없어요
#     # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
#     tokenizer = tf.keras.preprocessing.text.Tokenizer(
#         num_words=12000,  
#         filters=' ',
#         oov_token="<unk>"
#     )
#     # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다 - 문자 데이터 입력 받아 리스트 형태로 반환
#     tokenizer.fit_on_texts(corpus)
    
#     # 준비한 tokenizer를 이용해 corpus를 Tensor(시퀀스)로 변환합니다
#     tensor = tokenizer.texts_to_sequences(corpus)   
    
#     # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
#     # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
#     # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
#     tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

#     total_data_text = list(tensor)
#     num_tokens = [len(tokens) for tokens in total_data_text]
#     max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
#     maxlen = int(max_tokens)
#     # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
#     # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
#     tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=maxlen) # maxlen 사용하여 문장길이 제한 
    
#     print(tensor,tokenizer)
#     return tensor, tokenizer

# tensor, tokenizer = tokenize(corpus)

[[   2   50    5 ...    0    0    0]
 [   2   17 2639 ...    0    0    0]
 [   2   36    7 ...    0    0    0]
 ...
 [   2  130    5 ...    0    0    0]
 [ 287   79  162 ...  877  647    3]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fd6be305400>


In [132]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Now I've heard there was a secret chord
That David played, and it pleased the Lord
But you don't really care for music, do you?
It goes like this
The fourth, the fifth
The minor fall, the major lift
The baffled king composing Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah Your faith was strong but you needed proof


생성된 텐서 데이터 5행까지 출력해보기 

In [121]:
# tokenizer에 구축된 단어 사전의 인덱스 출력
print(tensor[:5, :]) 

[[   2   50    5   91  297   65   57    9  969 6042    3    0    0    0
     0    0    0    0    0    0]
 [   2   17 2639  873    4    8   11 6043    6  329    3    0    0    0
     0    0    0    0    0    0]
 [   2   36    7   37   15  164  282   28  299    4   47    7   43    3
     0    0    0    0    0    0]
 [   2   11  354   25   42    3    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [   2    6 3604    4    6 2265    3    0    0    0    0    0    0    0
     0    0    0    0    0    0]]


정제된 후 텐서, 문장 크기 출력해보기

In [129]:
print(len(tensor), len(corpus))

175749 175749


단어 사전이 어떻게 구축되었는지 확인해보기

In [123]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


 생성된 텐서를 소스와 타겟으로 분리하기

In [124]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2   50    5   91  297   65   57    9  969 6042    3    0    0    0
    0    0    0    0    0]
[  50    5   91  297   65   57    9  969 6042    3    0    0    0    0
    0    0    0    0    0]


텐서로 생성된 데이터를 이용해 tf.data.Dataset 객체 생성하기
- shuffle(), batch() 등 다양한 데이터셋 관련 기능 포함

In [125]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 19), (256, 19)), types: (tf.int32, tf.int32)>

### Step 4. 평가 데이터셋 분리

#### 총 데이터의 20% 를 평가 데이터셋으로 사용

In [126]:
from sklearn.model_selection import train_test_split

In [127]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                    tgt_input, 
                                                    test_size=0.2, 
                                                    random_state=42)

In [128]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 19)
Target Train: (140599, 19)


In [38]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [39]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 346, 7001), dtype=float32, numpy=
array([[[-6.3422485e-05, -4.8951997e-06,  3.5519279e-05, ...,
          1.4170856e-04, -2.1200298e-04,  3.1335570e-04],
        [-4.9308268e-04, -2.6116701e-04, -4.3154403e-04, ...,
          5.6984933e-04, -1.4284829e-04,  7.0104899e-04],
        [-6.0136931e-04, -2.4247836e-04, -1.0716727e-03, ...,
          9.2858647e-04, -2.2962558e-04,  8.4470509e-04],
        ...,
        [-5.5636931e-03, -2.4230885e-03, -1.7864021e-03, ...,
          3.4318154e-03,  1.0674139e-03,  3.3829203e-03],
        [-5.5636941e-03, -2.4230895e-03, -1.7864017e-03, ...,
          3.4318143e-03,  1.0674150e-03,  3.3829194e-03],
        [-5.5636941e-03, -2.4230888e-03, -1.7864020e-03, ...,
          3.4318136e-03,  1.0674152e-03,  3.3829189e-03]],

       [[-6.3422485e-05, -4.8951997e-06,  3.5519279e-05, ...,
          1.4170856e-04, -2.1200298e-04,  3.1335570e-04],
        [-5.5239016e-05, -3.0156266e-04, -9.3214316e-05, ...,
          7.1466598e-06, 